# ENTRY VIA JSON 

In [1]:
import os
import shutil
import re
from openpyxl import load_workbook
from datetime import datetime
from win32com import client
import num2words

import tkinter as tk
import configparser
from tkinter import ttk
from PIL import Image, ImageTk
from tkcalendar import DateEntry
import json
import multiprocessing
import queue
import threading
import os
import shutil
import re
from openpyxl import load_workbook
from datetime import datetime
from win32com import client
import num2words
import tkinter.messagebox as messagebox
import logging
import babel.numbers

def backend(message):
    """Process the JSON message and populate an Excel sheet with the extracted details."""

    try:
        print("lll")
        data = json.loads(message)
    

        # Extracting party details
        vendor = data['party_details']['party_name']

        # Extracting invoice details
        invoice_no = data['invoice_details']['invoice_number']
        date = data['invoice_details']['Invoice_date']

        # Validate date format
        format = "%d-%m-%Y"
        try:
            datetime.strptime(date, format)
        except ValueError:
            print (ValueError)
           
            return -1
        print("doing ...")

        # Copy the audit file to a new location

        config = configparser.ConfigParser()
        print("parsing done")

        # Path to the configuration file
        # Get the paths from the configuration file

        source = r"C:\Users\AMARJEET\Desktop\python_bill\AUDIT_FORMATE.xlsx"
        destination= r"C:\Users\AMARJEET\Desktop\python_bill\AUDIT\Audit"+invoice_no+".xlsx"

     
        shutil.copyfile(source, destination)

        # Load the workbook and access the active sheet
        workbook = load_workbook(filename=destination)
        sheet = workbook.active

        # Populate vendor and invoice details in the Excel sheet
        sheet["C2"] = vendor
        sheet["M2"] = vendor
        sheet["U2"] = vendor
        sheet["I3"] = invoice_no
        sheet["M3"] = invoice_no
        sheet["U3"] = invoice_no
        sheet["C3"] = date

        # Extract and populate item details from the JSON
        total_item = len(data['item_details'])
        for i in range(total_item):
            slno = int(data['item_details'][i]['sl_no'])
            description = data['item_details'][i]['description']
            HSN = int(data['item_details'][i]['hsn'])
            D_P = data['item_details'][i]['unit']
            if D_P == 'D':
                qty_purchased = int(data['item_details'][i]['quantity']) *12
                rate_purchased = round(float(data['item_details'][i]['rate']), 2) /12
            else:
                qty_purchased = int(data['item_details'][i]['quantity'])
                rate_purchased = round(float(data['item_details'][i]['rate']), 2)

            disc = round(float(data['item_details'][i]['dis']), 2)
            GST_RATE = round(float(data['item_details'][i]['gst']), 2)
            total_amount = round(float(data['item_details'][i]['total']), 2)

            cell_str_base = str(5 + slno)
            sheet[f"A{cell_str_base}"] = slno
            sheet[f"K{cell_str_base}"] = slno
            sheet[f"R{cell_str_base}"] = slno

            sheet[f"B{cell_str_base}"] = description
            sheet[f"G{cell_str_base}"] = HSN
            sheet[f"H{cell_str_base}"] = qty_purchased
            sheet[f"T{cell_str_base}"] = qty_purchased
            sheet[f"V{cell_str_base}"] = qty_purchased
            sheet[f"I{cell_str_base}"] = rate_purchased
            sheet[f"J{cell_str_base}"] = disc
            sheet[f"S{cell_str_base}"] = GST_RATE
            sheet[f"W{cell_str_base}"] = total_amount
            sheet[f"Y{cell_str_base}"] = round(float(total_amount * GST_RATE / 100), 2)
            sheet[f"U{cell_str_base}"] = 0

        # Save the workbook with populated details
        workbook.save(filename=destination)

    except json.JSONDecodeError as e:
        print(e)
        return -1
    except Exception as ex:
        print(ex)
      
        return -1
    return 1


#message = {"party_details": {"party_name": "EKTA AGENCIES", "party_address": "\n", "mobile_no": "", "email": "", "gstin": ""}, "invoice_details": {"invoice_number": "EKTA3413 ", "Invoice_date": "15-09-2024"}, "item_details": [{"sl_no": "1", "description": "DOMS HEXEL PEN BLUE ", "hsn": "96081019", "quantity": "200", "unit": "P", "rate": "2.330", "dis": "4", "gst": "18", "total": "447"}, {"sl_no": "2", "description": "DOMS Q 15 CM ", "hsn": "90172020", "quantity": "200", "unit": "P", "rate": "2.760", "dis": "4", "gst": "18", "total": "529"}, {"sl_no": "3", "description": "DOMS FSC 12 SHADES COLOUR  PENCIL ", "hsn": "96091000", "quantity": "20", "unit": "P", "rate": "39.290", "dis": "4", "gst": "12", "total": "754"}, {"sl_no": "4", "description": "DOMS WATER COLOUR PENS 12 SHADES MINI ", "hsn": "96082000", "quantity": "100", "unit": "P", "rate": "11.020", "dis": "4", "gst": "18", "total": "1057"}, {"sl_no": "5", "description": "DOMS DUST FREE ERASER", "hsn": "40169200", "quantity": "200", "unit": "P", "rate": "0.001", "dis": "0", "gst": "5", "total": "0"}, {"sl_no": "6", "description": "DOMS X1 PENCIL ", "hsn": "96091000", "quantity": "500", "unit": "P", "rate": "3.660", "dis": "4", "gst": "12", "total": "1756"}, {"sl_no": "7", "description": "DOMS REFILO WHITE BORAD MARKER BLUE ", "hsn": "96082000", "quantity": "5", "unit": "P", "rate": "0.001", "dis": "0", "gst": "18", "total": "0"}, {"sl_no": "8", "description": "DOMS KARBON PENCIL", "hsn": "96091000", "quantity": "500", "unit": "P", "rate": "4.640", "dis": "4", "gst": "12", "total": "2227"}, {"sl_no": "9", "description": "DOMS Q 15 CM ", "hsn": "90172020", "quantity": "50", "unit": "P", "rate": "0.001", "dis": "0", "gst": "18", "total": "0"}, {"sl_no": "10", "description": "DOMS OHP BLACK ", "hsn": "96082000", "quantity": "100", "unit": "P", "rate": "5.760", "dis": "4", "gst": "18", "total": "552"}, {"sl_no": "11", "description": "DOMS REFILO PERMANENT MARKER BLACK ", "hsn": "96082000", "quantity": "50", "unit": "P", "rate": "8.900", "dis": "4", "gst": "18", "total": "427"}, {"sl_no": "12", "description": "DOMS MINI E-RACER ERASER ", "hsn": "40169200", "quantity": "40", "unit": "P", "rate": "3.580", "dis": "4", "gst": "5", "total": "137"}, {"sl_no": "13", "description": "DOMS  E-RACER ERASER ", "hsn": "40169200", "quantity": "20", "unit": "P", "rate": "7.150", "dis": "4", "gst": "5", "total": "137"}]}
message = {"party_details": {"party_name": "EKTA AGENCIES ", "party_address": "\n", "mobile_no": "", "email": "", "gstin": ""}, "invoice_details": {"invoice_number": "EKTA_SMALL_OCT_NOV_1", "Invoice_date": "26-10-2024"}, "item_details": [{"sl_no": "1", "description": "28 PAGE SMALL GRAPH BOOK ", "hsn": "48209090", "quantity": "144", "unit": "", "rate": "5.36", "dis": "8", "gst": "12", "total": "710"}, {"sl_no": "2", "description": "DC 2 50 PAGE CHALLAN ", "hsn": "48201010", "quantity": "12", "unit": "", "rate": "44.76", "dis": "8", "gst": "18", "total": "505"}, {"sl_no": "3", "description": "A/4 SK BOOK SPIRAL", "hsn": "48202000", "quantity": "3", "unit": "", "rate": "26.790", "dis": "8", "gst": "12", "total": "73"}, {"sl_no": "4", "description": "A/6 SKETCH BOOK", "hsn": "48202000", "quantity": "3", "unit": "", "rate": "16.07", "dis": "8", "gst": "12", "total": "44"}, {"sl_no": "5", "description": "A/4 ATRIST BOOK SPIRAL ", "hsn": "48202000", "quantity": "2", "unit": "", "rate": "80.360", "dis": "8", "gst": "12", "total": "147"}, {"sl_no": "6", "description": "28 PAGE SAMLL GRAPH_BOOK", "hsn": "48209090", "quantity": "72", "unit": "", "rate": "5.360", "dis": "8", "gst": "12", "total": "355"}, {"sl_no": "7", "description": "BLUE BALL PEN XO", "hsn": "960810", "quantity": "1000", "unit": "", "rate": "5.93", "dis": "4", "gst": "18", "total": "5692"}, {"sl_no": "8", "description": "28 PAGE SMALL GRAPH BOOK", "hsn": "48209090", "quantity": "144", "unit": "", "rate": "5.36", "dis": "8", "gst": "12", "total": "710"}, {"sl_no": "9", "description": "A/4 BOOK 172 PAGE", "hsn": "48202000", "quantity": "36", "unit": "", "rate": "32.14", "dis": "0", "gst": "12", "total": "1160"}, {"sl_no": "10", "description": "H-XO BALL PEN BLUE", "hsn": "960810", "quantity": "1600", "unit": "", "rate": "5.93", "dis": "4", "gst": "18", "total": "9108"}, {"sl_no": "11", "description": "H-XO BALL PEN  BLACK", "hsn": "960810", "quantity": "600", "unit": "", "rate": "5.93", "dis": "4", "gst": "18", "total": "3415"}, {"sl_no": "12", "description": "H-XO BALL PEN  RED", "hsn": "960810", "quantity": "200", "unit": "", "rate": "5.93", "dis": "4", "gst": "18", "total": "1138"}, {"sl_no": "13", "description": "H-XO BALL PEN  CARD HAMMER", "hsn": "960810", "quantity": "2", "unit": "", "rate": "0.01", "dis": "0", "gst": "18", "total": "0"}, {"sl_no": "14", "description": "H-XO  GEL PEN BLUE", "hsn": "960810", "quantity": "200", "unit": "", "rate": "5.93", "dis": "4", "gst": "18", "total": "1138"}, {"sl_no": "15", "description": "50 PAGE D CHALLAN BOOK ", "hsn": "48201010", "quantity": "12", "unit": "", "rate": "45.76", "dis": "8", "gst": "18", "total": "505"}, {"sl_no": "16", "description": "H-GOLD POINT BALL PEN", "hsn": "960810", "quantity": "200", "unit": "", "rate": "6.140", "dis": "4", "gst": "18", "total": "1178"}, {"sl_no": "17", "description": "XO GEL BLUE PEN ", "hsn": "960810", "quantity": "200", "unit": "", "rate": "5.93", "dis": "4", "gst": "18", "total": "1138"}, {"sl_no": "18", "description": "DOMS TEMPRA COLOR S 12 ", "hsn": "32131000", "quantity": "30", "unit": "", "rate": "12.711", "dis": "4", "gst": "18", "total": "366"}, {"sl_no": "19", "description": "DOMS PAINTING KIT", "hsn": "32131000", "quantity": "20", "unit": "", "rate": "126.48", "dis": "4", "gst": "18", "total": "2428"}, {"sl_no": "20", "description": "DOMS ART STOKES KIT ", "hsn": "91091000", "quantity": "20", "unit": "", "rate": "100", "dis": "4", "gst": "12", "total": "1920"}, {"sl_no": "21", "description": "DOMS GEOFINE MATHS BOX", "hsn": "90172020", "quantity": "20", "unit": "", "rate": "44.06", "dis": "4", "gst": "18", "total": "847"}, {"sl_no": "22", "description": "DOMS PENCIL SMART KIT ", "hsn": "90172020", "quantity": "2", "unit": "", "rate": "317.79", "dis": "4", "gst": "18", "total": "610"}, {"sl_no": "23", "description": "DOMS NEO PRIME KIT ", "hsn": "96081019", "quantity": "37", "unit": "", "rate": "1.53", "dis": "0", "gst": "18", "total": "56"}, {"sl_no": "24", "description": "TWO SIDE LAB SHEET ", "hsn": "4820200", "quantity": "20", "unit": "", "rate": "29.46", "dis": "8", "gst": "12", "total": "542"}, {"sl_no": "25", "description": "DOMS OIL PASTEL 12 SET", "hsn": "96099030", "quantity": "20", "unit": "", "rate": "25", "dis": "4", "gst": "12", "total": "480"}, {"sl_no": "26", "description": "DOMS BRUSH PEN", "hsn": "96082000", "quantity": "5", "unit": "", "rate": "118.64", "dis": "4", "gst": "18", "total": "570"}, {"sl_no": "27", "description": "DOMS CARBON PENCIL", "hsn": "96091000", "quantity": "200", "unit": "", "rate": "4.64", "dis": "4", "gst": "12", "total": "890"}, {"sl_no": "28", "description": "DOMS NEON ERASER", "hsn": "40169200", "quantity": "60", "unit": "", "rate": "3.57", "dis": "4", "gst": "5", "total": "205"}, {"sl_no": "29", "description": "DOMS Q 30 CM", "hsn": "90172020", "quantity": "100", "unit": "", "rate": "7.2", "dis": "4", "gst": "18", "total": "691"}, {"sl_no": "30", "description": "DOMS WAX CRYONS 10", "hsn": "96091000", "quantity": "100", "unit": "", "rate": "5.36", "dis": "4", "gst": "12", "total": "514"}, {"sl_no": "31", "description": "DOMS WAX CRYOON 12 ", "hsn": "96091000", "quantity": "50", "unit": "", "rate": "7.59", "dis": "4", "gst": "12", "total": "364"}, {"sl_no": "32", "description": "DOMS WATER COLOR PEN", "hsn": "96082000", "quantity": "100", "unit": "", "rate": "11.02", "dis": "4", "gst": "18", "total": "1057"}, {"sl_no": "33", "description": "DOMS GEOMMY MATH DRAW INST", "hsn": "90172020", "quantity": "40", "unit": "", "rate": "53.39", "dis": "4", "gst": "18", "total": "2050"}, {"sl_no": "34", "description": "DOMS NEON R/T PENCIL ", "hsn": "96091000", "quantity": "500", "unit": "", "rate": "4.018", "dis": "4", "gst": "12", "total": "1928"}, {"sl_no": "35", "description": "DOMS X1 PENCIL", "hsn": "96091000", "quantity": "300", "unit": "", "rate": "3.66", "dis": "4", "gst": "12", "total": "1054"}, {"sl_no": "36", "description": "DOMS HEXEL PEN BLUE ", "hsn": "96081019", "quantity": "200", "unit": "", "rate": "2.33", "dis": "4", "gst": "18", "total": "447"}, {"sl_no": "37", "description": "DOMS REFILO WHITE BOARD MARKER BLACK", "hsn": "96082000", "quantity": "50", "unit": "", "rate": "10.59", "dis": "4", "gst": "18", "total": "508"}, {"sl_no": "38", "description": "DOMS MY FIRST PENCIL KIT", "hsn": "90172020", "quantity": "50", "unit": "", "rate": "8.48", "dis": "4", "gst": "18", "total": "407"}, {"sl_no": "39", "description": "DOMS TEMPRA COLOR", "hsn": "32131000", "quantity": "36", "unit": "", "rate": "12.711", "dis": "4", "gst": "18", "total": "440"}, {"sl_no": "40", "description": "DOMS LONG COLORED ERASER", "hsn": "40169200", "quantity": "200", "unit": "", "rate": "1.38", "dis": "4", "gst": "5", "total": "264"}, {"sl_no": "41", "description": "DOMS EURO WHITE ERASER", "hsn": "40169200", "quantity": "200", "unit": "", "rate": "0.67", "dis": "4", "gst": "5", "total": "128"}, {"sl_no": "42", "description": "DOMS HEXEL PEN BLUE", "hsn": "96081019", "quantity": "200", "unit": "", "rate": "2.33", "dis": "4", "gst": "18", "total": "450"}, {"sl_no": "43", "description": "DOMS X1 PENCIL", "hsn": "96091000", "quantity": "500", "unit": "", "rate": "3.06", "dis": "4", "gst": "12", "total": "1760"}, {"sl_no": "44", "description": "DOMS REFILO WHITE BOARD MARKER BLACK", "hsn": "96082000", "quantity": "5", "unit": "", "rate": "0.01", "dis": "4", "gst": "18", "total": "0"}, {"sl_no": "45", "description": "DOMS WATER COLOR PEN 12 ", "hsn": "9608200", "quantity": "100", "unit": "", "rate": "11.02", "dis": "4", "gst": "18", "total": "1057"}, {"sl_no": "46", "description": "DOMS MINI ERASER", "hsn": "40169200", "quantity": "80", "unit": "", "rate": "3.58", "dis": "4", "gst": "5", "total": "274"}, {"sl_no": "47", "description": "12 TRIP PENCIL COLOR ", "hsn": "96091000", "quantity": "20", "unit": "", "rate": "12.5", "dis": "4", "gst": "12", "total": "240"}, {"sl_no": "48", "description": "DOMS WAX CRYONS", "hsn": "96091000", "quantity": "100", "unit": "", "rate": "5.36", "dis": "4", "gst": "12", "total": "514"}, {"sl_no": "49", "description": "DOMS WAX PEN ", "hsn": "96082000", "quantity": "5", "unit": "", "rate": "118.64", "dis": "4", "gst": "18", "total": "569"}]}
backend(json.dumps(message))
# Ensure to define and use the message queue in your main program.

lll
doing ...
parsing done


1